In [1]:
from PIL import Image
import os
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
IMAGE_SIZES = [100, 100]

In [3]:
# reads all images in filepath, resizes it according with 2D list img_sizes and 
# store them in a np_array. The string filepath can only contain images (no other
# file types such as txt for example are allowed). 
# It works only for RGB images.
def images_to_np_array(filepath, img_sizes):
    #-----------------------------------------------------------------------------
    # Args:
    #     filepath:  String with the path to the directory containing the images.
    #                It CANNOT end with the / character.
    #     img_sizes: 2D array [num_rows, num_columns].
    #-----------------------------------------------------------------------------

    # list all files in filepath
    list_of_files = [file for file in os.listdir(filepath)]

    # number of image files
    n_images = len(list_of_files)

    # create np_array to store all images
    array_images = np.zeros([n_images, IMAGE_SIZES[0], IMAGE_SIZES[1], 3])

    for index in range( n_images ):

        # get file name   
        file_name = list_of_files[index]

        # Open the image form working directory
        image = Image.open(filepath + '/' + file_name)

        # resize image
        image_resized = image.resize(IMAGE_SIZES)

        # convert it to numpy
        np_image = np.asarray(image_resized)
        array_images[index,:,:,:] = np_image

    return array_images



In [4]:
# read all capivaras
capivara_array = images_to_np_array(filepath = "./capivara", img_sizes = IMAGE_SIZES)
y_capivara = np.zeros(capivara_array.shape[0])

# read all dunkies
burro_array = images_to_np_array(filepath = "./burro", img_sizes = IMAGE_SIZES)
y_burro = np.ones(burro_array.shape[0])

# create y vector with labels
y = np.concatenate( (y_capivara, y_burro) )

# concatenate and shuffle images
array_images = np.concatenate( (capivara_array, burro_array), axis = 0 )
number_of_images = array_images.shape[0]
permuted_index = np.random.permutation(number_of_images)
array_images = array_images[permuted_index, :, :, :]
y = y[permuted_index]

# splitting training and test sets

x_train = array_images[0:700,:,:,:]
y_train = y[0:700]

x_test = array_images[700:,:,:,:]
y_test = y[700:]


In [5]:
# preprocessing 
array_images = tf.keras.applications.densenet.preprocess_input(array_images, data_format = "channels_last")

In [6]:

# initialize model with weights treined in imagenet 
base_model = tf.keras.applications.DenseNet169(
    include_top=False,
    weights="imagenet",
    input_shape=(IMAGE_SIZES[0], IMAGE_SIZES[1], 3),
    pooling="avg",
    classes=2,
    classifier_activation="None"
)

# Freeze model weights
base_model.trainable = False

2022-08-14 21:04:44.150034: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 21:04:44.193016: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 21:04:44.193282: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 21:04:44.193960: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [7]:
data_augmentation = tf.keras.Sequential(
    [tf.keras.layers.RandomFlip("horizontal"), tf.keras.layers.RandomRotation(0.5),]
)

In [8]:
inputs = tf.keras.Input(shape=(IMAGE_SIZES[0], IMAGE_SIZES[1], 3))
# apply data augmentation
x = data_augmentation(inputs)
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(x, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = tf.keras.layers.Flatten(data_format = "channels_last")(x)
x = tf.keras.layers.Dense(100, activation = "relu")(x)
x = tf.keras.layers.Dense(100, activation = "relu")(x)
x = tf.keras.layers.Dense(100, activation = "relu")(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 100, 100, 3)       0         
                                                                 
 densenet169 (Functional)    (None, 1664)              12642880  
                                                                 
 flatten (Flatten)           (None, 1664)              0         
                                                                 
 dense (Dense)               (None, 100)               166500    
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100 

In [18]:
#checkpoint_filepath = "./trained_model_checkpoint"

# define the callback (to save the weights that gave best val_accuracy during training)
#callback = tf.keras.callbacks.ModelCheckpoint(monitor='val_accuracy',
#                                              mode = "max",
#                                              save_best_only = True,
#                                              filepath = checkpoint_filepath)


In [19]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy()],
)

history = model.fit(x = x_train, 
                    y = y_train, 
                    epochs = 10, 
                    validation_data = (x_test, y_test), 
                    batch_size = 100)


Epoch 1/10
7/7 [==============================] - ETA: 0s - loss: 0.0151 - binary_accuracy: 0.9943 - accuracy: 0.9943

In [15]:
model.evaluate(y = y_test, x = x_test, verbose = 2)

7/7 - 0s - loss: 0.1863 - binary_accuracy: 0.9559 - 394ms/epoch - 56ms/step


[0.1863396018743515, 0.9558823704719543]

In [16]:
predictions = model.predict(x_test)
pd.DataFrame( {"probability": tf.keras.activations.sigmoid(predictions)[:,0], "label": y_test})

7/7 [==============================] - 2s 56ms/step


,probability,label
0,9.999971e-01,1.0
1,1.236003e-04,0.0
2,1.428722e-12,0.0
3,2.243708e-08,0.0
4,5.920243e-11,0.0
...,...,...
199,9.999963e-01,1.0
200,3.788940e-07,0.0
201,8.804154e-02,1.0
202,1.000000e+00,1.0


In [ ]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy()],
)

epochs = 10
model.fit(x = x_train, y = y_train, epochs=epochs, validation_data = (x_test, y_test))